In [1]:
from urllib.request import urlopen as uReq
import re
import requests
import bs4
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
from os.path import basename

In [2]:
width_dict = {"max-width:7%;": "1", "max-width:14%;": "2", "max-width:21%;": "3", 
              "max-width:28%;": "4", "max-width:35%;": "5"} #1 being least 5 being most

In [3]:
gender_dict = {"Homme": "Male", "Femme": "Female"}

In [4]:
disease_dict = {"Acné": "Acne", "Allergie":"Allergy", "Allergie acariens":"Dust mite allergy", "Asthme":"Asthma",
               "Ampoules - cloques":"Blisters - blisters", "Angine de poitrine":"Angina pectoris",
               "Angoisse / trouble panique":"Anxiety", "Arrêter de fumer":"Stop smoking", "Bronchite":"Bronchitis",
               "Arthrite psoriatique":"Psoriatic arthritis", "Arthrose":"Osteoarthritis", "Brûlures":"Burns",
               "Arthrose colonne vertébrale":"Spine osteoarthritis", "Cancer du sein":"Breast cancer",
               "Céphalées":"Headache", "Cholestérol élevé":"High cholesterol", "Couperose":"Rosacea",
               "Colite ulcéreuse":"Ulcerative colitis", "Crise cardiaque":"Heart attack", "Tremblements":"Tremors",
               "Complications post-opératoires":"Post-operative complications", "Démangeaison":"Itching",
               "Cycle menstruel irrégulier":"Irregular menstrual cycle", "Défaillance nerveuse":"Nerve failure",
               "Dépression":"Depression", "Dépression post-natale":"Post-natal depression", "Thyroïde":"Thyroid",
               "Dépression chronique":"Depression", "Diabète type 1":"Type 1 diabetes", "TDAH":"ADHD",
               "Diabète type 2":"Type 2 diabetes", "Diabète type 3":"Type 3 diabetes", "Urticaire":"Urticaria",
               "Troubles d'angoisse":"Anxiety", "Trouble stress post-traumatique":"Post-traumatic stress disorder",
               "Trouble schizotypique":"Schizotypal disorder", "Trouble rythme cardiaque":"Heart rhythm disorder",
               "Trouble obsessionnel compulsif (TOC)":"Obsessive Compulsive Disorder (OCD)", "Psychose":"Psychosis",
               "Trouble bipolaire / trouble mani dépressif":"Bipolar disorder / manic depressive disorder",
               "Thyroïde enlevée":"Thyroid removed", "Tension élevée":"High tension", "Sinusite":"Sinusitis",
               "Spondylarthrite ankylosante":"Ankylosing spondylitis", "Schizophrénie":"Schizophrenia",
               "Rhume des foins":"Hay fever", "Rhumatisme":"Rheumatism", "Retour d'âge":"Return of age",
               "Pyélonéphrite":"Pyelonephritis", "Problèmes psychiques":"Psychic problems", "Otite":"Otitis",
               "Problèmes cardiaques":"Heart problems", "Pouls élevé":"High pulse", "Pneumonie":"Pneumonia",
               "Polyneuropathie":"Polyneuropathy", "Phobie sociale":"Social phobia", "Narcolepsie":"Narcolepsy",
               "MST (maladies sexuellement transmissibles)":"STDs (sexually transmitted diseases)",
               "Mise en place stérilet":"IUD insertion", "Maladie de Lyme - borréliose":"Lyme disease - borreliosis",
               "Maladie de Graves":"Graves disease", "Maladie de Crohn":"Crohn's disease", "Insomnie":"Insomnia",
               "Maladie d'Hashimoto":"Hashimoto's disease", "Maladie auto-immunitaire":"Autoimmune disease",
               "Mal de gorge":"Sore throat", "Inflammation articulaire":"Joint inflammation", "Hernie":"Hernia",
               "Infection urinaire":"Urinary tract infection", "Infection gencive":"Gum infection", "Fatigue":"Tired",
               "Infection des amygdales":"Infection of the tonsils", "Infection de la prostate":"Prostate infection",
               "Infection de la machoire":"Jaw infection", "Infection de l'estomac":"Stomach infection",
               "Infection dans la poitrine":"Chest Infection", "Infection bactérienne":"Bacterial infection",
               "Infection (plaie)":"Infection (wound)", "Glande thyroïde lente":"Slow thyroid gland",
               "Glande thyroïde hyperactive":"Overactive thyroid gland", "Fibromyalgie":"Fibromyalgia",
               "Épisode maniaque":"Manic episode", "Épilepsie":"Epilepsy", "Douleur, général":"Pain, general",
               "Douleurs menstruation":"Menstruation pain", "Douleur articulaire":"Articular pain",
               "Douleur nerveuse":"Nerve pain", "Hypothyroïdie":"Hypothyroidism", "Battements de coeur": "Heartbeats",
               "Ablation de la thyroïde":"Thyroid removal", "Pas dans la liste":"Other", "Apnée":"Apnea",
               "Tendinite épaule":"Shoulder tendinitis", "Alcoolisme":"Alcoholism", "Mal aux dents":"Toothache",
               "Syndrome des jambes sans repos":"Restless legs syndrome", "Vertige - tournis":"Vertigo - spinning",
               "Cholestérol trop élevé":"High cholesterol","Crampes jambes":"Leg cramps", "Acidité":"Acidity",
               "Effet indésirable d'un autre médicament":"Side effect of another medicine", "Attaque":"Attack",
               "Anémie suite déficience vitamine B12":"Anemia", "Défaillances cardiaques":"Cardiac failures",
               "Brûlure d'estomac - acidité gastrique":"Heartburn - gastric acidity"}

In [5]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [6]:
#We collect all pages for Levothyrox

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes third value in var
        match = rx_dose.search(subText[3]) #takes fourth element of subText
        row['dose'] = np.nan if match is None else match.group(1)
        row['disease'] = subText[4].strip()
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [21]:
url = 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox'

def collect_data(url):
    disease = basename.url
    urls = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [18]:
urls = [url] + [url + '/' + str(i) for i in range(2, 8)]
urls

['https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox',
 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox/2',
 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox/3',
 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox/4',
 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox/5',
 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox/6',
 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox/7']

In [22]:
rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes third value in var
        match = rx_dose.search(subText[3]) #takes fourth element of subText
        row['dose'] = np.nan if match is None else match.group(1)
        row['disease'] = subText[4].strip()
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

MissingSchema: Invalid URL 'h': No schema supplied. Perhaps you meant http://h?

In [ ]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [ ]:
df
#f"Population Pyramid for {medicine}"

In [ ]:
df.to_csv('test.csv') #we store the dataframe in a csv file